In [1]:
import hail as hl

In [2]:
hl.init()

2022-12-01 12:50:47 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-12-01 12:50:48 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running on Apache Spark version 3.1.3
SparkUI available at http://192.168.0.181:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.99-57537fea08d4
LOGGING: writing to /Users/adrian/BroadIS/hail-20221201-1250-0.2.99-57537fea08d4.log


In [11]:
maps_table = hl.read_table(
'gs://janucik-dataproc-stage/01_maps/v1_30_Nov_2022_MAPS_per_gene_using_arrays/maps_downsampling_multi_model.ht'
)


In [12]:
maps_table.show(4)

+-------------------+----------------------+
| gene_ids          | lof_csq_collapsed    |
+-------------------+----------------------+
| str               | str                  |
+-------------------+----------------------+
| "ENSG00000000003" | "HC"                 |
| "ENSG00000000003" | "missense_variant"   |
| "ENSG00000000003" | "synonymous_variant" |
| "ENSG00000000005" | "HC"                 |
+-------------------+----------------------+

+------------------------------------------------------------------------------+
| N_singletons                                                                 |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,... |
| [42,40,19,0,1,2,0,0,15,7,8,4,38,17,16,1,1,0,0,0,14,6,7,6,6,1,0,2,0,1,8,1,... |
| [9,8,7,0,1,1,0,0,2,1,1,4,8,7,5,0,0,1,0,0,2,1,0,3,4,0,1,0,0,0,1,0,1,4,0,0,... |
| [1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| expected_singletons                                                          |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [5.67e-01,5.59e-01,6.12e-01,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,... |
| [3.26e+01,3.21e+01,1.49e+01,1.10e+00,7.23e-01,9.55e-01,8.03e-02,4.24e-01,... |
| [1.23e+01,1.21e+01,7.30e+00,3.66e-01,3.63e-01,9.57e-01,0.00e+00,2.32e-01,... |
| [1.64e-01,1.58e-01,3.51e-01,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| N_variants                                                                   |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,... |
| [65,65,27,4,2,3,1,3,26,7,11,9,55,28,23,3,2,1,1,3,23,6,8,8,11,4,1,3,1,3,11... |
| [25,25,13,1,1,2,0,1,7,3,1,8,21,15,11,1,0,2,0,1,6,3,0,6,5,1,1,0,0,1,5,0,1,... |
| [1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| ps_agg                                                                       |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [1.00e+00,1.00e+00,1.00e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00e+00,... |
| [6.46e-01,6.15e-01,7.04e-01,0.00e+00,5.00e-01,6.67e-01,0.00e+00,0.00e+00,... |
| [3.60e-01,3.20e-01,5.38e-01,0.00e+00,1.00e+00,5.00e-01,NaN,0.00e+00,2.86e... |
| [1.00e+00,1.00e+00,1.00e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00e... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| maps                                                                         |
+------------------------------------------------------------------------------+
| array<float64>                                              

In [13]:
ht = hl.read_table('gs://gcp-public-data--gnomad/release/3.1.2/ht/genomes/gnomad.genomes.v3.1.2.sites.ht')
meta_fields = ht.freq_meta.collect()
maps_table = maps_table.annotate_globals(freq_metas = meta_fields)

In [14]:
maps_zipped = maps_table.transmute(
data = hl.zip(maps_table.N_singletons, maps_table.expected_singletons, maps_table.N_variants, maps_table.ps_agg, maps_table.maps, maps_table.maps_sem, maps_table.freq_metas[0])
)

In [15]:
maps_expl = maps_zipped.explode('data')

In [16]:
maps_clean = maps_expl.annotate(
Singletons = maps_expl.data[0],
Expected_Singletons = maps_expl.data[1],
N_variants = maps_expl.data[2],
PS = maps_expl.data[3],
MAPS = maps_expl.data[4],
MAPS_Sem = maps_expl.data[5],
Metadata = maps_expl.data[6].values()
)

In [19]:
maps_clean_global =  maps_clean.filter(
    (maps_clean.Metadata[0] == 'adj') & (maps_clean.Metadata.length() == 1))

maps_clean_global = maps_clean_global.annotate(
    population = maps_clean_global.Metadata[0])

maps_clean_global = maps_clean_global.select('Singletons', 'Expected_Singletons', 'N_variants', 'PS', 'MAPS', 'MAPS_Sem', 'population')

maps_clean_global.export('/Users/adrian/BroadIS/MAPS/01_data/maps_clean_global_Per_gene.csv', delimiter = ",")

maps_clean_global.show()


2022-12-01 12:54:31 Hail: INFO: merging 105 files totalling 781.5M...+ 7) / 104]
2022-12-01 12:54:33 Hail: INFO: while writing:                    (8 + 8) / 104]
    /Users/adrian/BroadIS/MAPS/01_data/maps_downsampling_multi_model_Per_Gene.csv
  merge time: 2.032s
2022-12-01 12:55:15 Hail: INFO: merging 105 files totalling 4.9M...2 + 2) / 104]
2022-12-01 12:55:15 Hail: INFO: while writing:
    /Users/adrian/BroadIS/MAPS/01_data/maps_clean_global_Per_gene.csv
  merge time: 48.507ms


,,,,,,,,
gene_ids,lof_csq_collapsed,Singletons,Expected_Singletons,N_variants,PS,MAPS,MAPS_Sem,population
str,str,int64,float64,int64,float64,float64,float64,str
"""ENSG00000000003""","""HC""",1,5.67e-01,1,1.00e+00,4.33e-01,0.00e+00,"""adj"""
"""ENSG00000000003""","""missense_variant""",42,3.26e+01,65,6.46e-01,1.45e-01,5.93e-02,"""adj"""
"""ENSG00000000003""","""synonymous_variant""",9,1.23e+01,25,3.60e-01,-1.31e-01,9.60e-02,"""adj"""
"""ENSG00000000005""","""HC""",1,1.64e-01,1,1.00e+00,8.36e-01,0.00e+00,"""adj"""
"""ENSG00000000005""","""missense_variant""",30,2.77e+01,57,5.26e-01,4.05e-02,6.61e-02,"""adj"""
"""ENSG00000000005""","""synonymous_variant""",14,1.46e+01,30,4.67e-01,-2.02e-02,9.11e-02,"""adj"""
"""ENSG00000000419""","""HC""",6,4.31e+00,9,6.67e-01,1.88e-01,1.57e-01,"""adj"""
"""ENSG00000000419""","""missense_variant""",62,5.10e+01,102,6.08e-01,1.08e-01,4.83e-02,"""adj"""
